## Prepare / Setup

### Export Maple Data and Environment Variables

In [1]:
import yaml
import os
from etl.prepare import *

with open('./config.yaml', 'r') as file:
    config_data = yaml.safe_load(file)

In [ ]:
import redshift_connector
from etl.postgresqlschemareader import *

with redshift_connector.connect(
    host=config_data['ams']['host'],
    database=config_data['ams']['database'],
    user=config_data['ams']['username'],
    password=config_data['ams']['password'],
    timeout=999999,
    port=5439
) as conn:
    with conn.cursor() as cur:
        cur.execute("""SELECT table_schema, table_name
                      FROM information_schema.tables
                      WHERE table_schema != 'pg_catalog'
                      AND table_schema != 'information_schema'
                      AND table_type='BASE TABLE'
                      ORDER BY table_schema, table_name""")

        tables = cur.fetchall()
        print(tables)

## Extract required data

In [2]:
from etl.extract import *
from etl.transform import *

In [19]:
batch_num = '1'

with postgresql_conn(params = config_data['postgresql_prod']) as conn:
    conn.autocommit = True
    with conn.cursor() as cur:
        # student_participants = extract_student_participants(filepath = '../maple-s3/participants_post.xlsx')
        cur.execute("SELECT * FROM maple.isot_table")
        isot_table = pd.DataFrame(cur.fetchall()).drop_duplicates('isocntcd',keep = 'last')
        cur.execute("SELECT * FROM maple.maple_student_post_val")
        student_participants_post = pd.DataFrame(cur.fetchall())
        student_participants_post['username'] = student_participants_post['username'].astype(str)
        student_participants_post['isocntcd'] = student_participants_post['username'].str.slice(1,4)

        countries_all = isot_table.isoalpha3.unique()
        countries_now = list(student_participants_post.loc[student_participants_post['batch'] == batch_num,:].isoalpha3.unique())
        countries_post = list(student_participants_post.loc[student_participants_post['batch'] != batch_num,:].isoalpha3.unique())
        countries_pre_init = list(set(countries_all) - set(countries_now) - set(countries_post))

        student_participants_pre = pd.read_excel('../maple-s3/participants_with_entity.xlsx').drop_duplicates(['username'],keep = 'last')
        student_participants_pre['username'] = student_participants_pre['username'].astype(str)
        student_participants_pre['isocntcd'] = student_participants_pre['username'].str.slice(1,4)
        student_participants_pre = student_participants_pre.loc[student_participants_pre['isoalpha3'].isin(countries_pre_init)].drop_duplicates(subset = ['username'],keep = 'last')
        student_participants_pre['login'] = student_participants_pre['username']
        student_participants_pre = student_participants_pre.rename({'testAttendance':'test_attendance','questionnaireAttendance': 'questionnaire_attendance'},axis = 1)
        countries_pre = list(set(student_participants_pre.isoalpha3.unique()) - set(['GBR']))

        student_participants = pd.concat(
            [
                student_participants_pre,
                student_participants_post
            ],
            axis = 0
        )
        
        nc_dat_now = isot_table.loc[isot_table['isoalpha3'].isin(countries_now)].assign(process='now')
        nc_dat_post = isot_table.loc[isot_table['isoalpha3'].isin(countries_post)].assign(process='post')
        nc_dat_pre = isot_table.loc[isot_table['isoalpha3'].isin(countries_pre)].assign(process='pre')  
        nc_dat = pd.concat(
            [
                nc_dat_post,
                nc_dat_now,
                nc_dat_pre
            ],
            axis = 0
        )
        # nc_dat = extract_country_codes(filepath = './data/maple-s3/ISOT_table.xlsx')

In [72]:
import sys, importlib
importlib.reload(sys.modules['etl.load'])

<module 'etl.load' from 'd:\\Users\\leon.head\\Documents\\pisa2025-api-etl\\etl\\load.py'>

In [20]:
import glob, time

count = 0
domain_all = ["FLA"]

for domain in domain_all:
    cbk = create_codebook(domain = domain)
    
    if(domain == 'FLA'):
        gap_vars = True

    for idr,row in nc_dat.iterrows():
        start_time = time.time()
        country_print = str(row['isoalpha3'])
        print(f"Processing data for: {country_print}")

        extract_json(domain = domain, nc_dat = row ,overwrite = False, con = postgresql_conn(params = config_data['postgresql_prod']))

        filepath = f"./data/db/{domain.lower()}/{domain}_{country_print}.json"

        if(os.path.isfile(filepath)):
            df = read_json_file(filepath)
            print("Step 1: rows = " + str(df.shape[0]) + ' & columns = ' + str(df.shape[1]))
            
            df1 = explode_raw_data(df = df)
            print("Step 2: rows = " + str(df1.shape[0]) + ' & columns = ' + str(df1.shape[1]))

            df3 = explode_items(df1)
            print("Step 3: rows = " + str(df3.shape[0]) + ' & columns = ' + str(df3.shape[1]))

            df4 = explode_values(df3)
            df4 = rename_variables(df4, domain = domain)
            df4 = check_duplicates(df4)
            df4 = replace_blank_json(df4)
            # print("Step 4: rows = " + str(df4.shape[0]) + ' & columns = ' + str(df4.shape[1]))

            df6 = explode_responses(df4, domain = domain)
            if(domain == 'FLA'):
                df6 = fla_recode_FLALDTB1002(df6)
            if(gap_vars):
                df6 = gap_recode(df6,cbk)
            print("Step 5: rows = " + str(df6.shape[0]) + ' & columns = ' + str(df6.shape[1]))

            df8 = merge_cbk_status(df6,cbk,domain = 'FLA')
            df8 = time_var_recode(df8)
            df8 = score_resp_recode(df8,domain = 'FLA') 
            df8 = trailing_missing(df8,cbk=cbk)
            df8 = cmc_item_create(df8,cbk=cbk, domain = 'FLA')
            df9 = merge_participant_info(df8,student_participants=student_participants)
            df9['ppart1'] = df9['ppart1'].astype(str).apply(lambda x: re.sub(".0","",x)).replace('nan','')
            df9['mpop1'] = df9['mpop1'].astype(str).apply(lambda x: re.sub(".0","",x)).replace('nan','')
            print("Step 6: rows = " + str(df9.shape[0]) + ' & columns = ' + str(df9.shape[1]))

            df_resp_check = sql_query_ge(nc_dat = row,cbk = cbk,con = postgresql_conn(params = config_data['postgresql_prod']))

            if(count == 0):
                df_long = df9
                df_long_check = df_resp_check
            else:
                df_long = pd.concat([df_long,df9],axis = 0)
                df_long_check = pd.concat([df_long_check,df_resp_check],axis = 0)
            
            count =+ 1

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Time taken for {filepath}: {elapsed_time:.2f} seconds")

        # df9.export_to_postgresql()

Extracting codebook data from sheet FLA_Reading_CQ
Extracting codebook data from sheet FLA_Listening_CQ
Codebook created for FLA
Processing data for: AUT
Connection to DB established, searching for new records...
Connection to DB closed
Processing data for: BRN
Connection to DB established, searching for new records...
Connection to DB closed
Processing data for: QCY
Connection to DB established, searching for new records...
Connection to DB closed
Step 1: rows = 898 & columns = 9
Step 2: rows = 898 & columns = 33
Step 3: rows = 20965 & columns = 8
Step 5: rows = 61523 & columns = 22
Step 6: rows = 62245 & columns = 50
Extracting SQL query checks...
Time taken for ./data/db/fla/FLA_QCY.json: 86.12 seconds
Processing data for: DEU
Connection to DB established, searching for new records...
Connection to DB closed
Step 1: rows = 1240 & columns = 9
Step 2: rows = 1240 & columns = 33
Step 3: rows = 28950 & columns = 8
Step 5: rows = 84868 & columns = 22
Step 6: rows = 85921 & columns = 50
E

In [21]:
from test.data_quality.DataQuality import DataQuality
from test.utils.utils import create_df_from_dq_results

df_check = df_long.loc[df_long['in_cq'] == '1',:]
df_check.head()

,index,login,last_update_date,testQtiLabel,sessionStartTime,sessionEndTime,language,unit_id,itemId,score,...,dob_yy,sen,mpop1,ppart1,isocntcd,isoalpha3,isoname,isocntcd,test_attendance,questionnaire_attendance
12,12.0,11960044002,1.720221e+12,FLA-L-9,2024-04-03 17:19:28,2024-04-03 17:44:21,en-ZZ,FLAL5IMCB1001,cluster1-FLAL09-item-2,1,...,2008,0,1,1,196,QCY,Cyprus,196,1,1
13,13.0,11960044002,1.720221e+12,FLA-L-9,2024-04-03 17:19:28,2024-04-03 17:44:21,en-ZZ,FLAL5IMCB1001,cluster1-FLAL09-item-2,1,...,2008,0,1,1,196,QCY,Cyprus,196,1,1
15,15.0,11960044002,1.720221e+12,FLA-L-9,2024-04-03 17:19:28,2024-04-03 17:44:21,en-ZZ,FLAL5IMCB1001,cluster1-FLAL09-item-2,1,...,2008,0,1,1,196,QCY,Cyprus,196,1,1
16,16.0,11960044002,1.720221e+12,FLA-L-9,2024-04-03 17:19:28,2024-04-03 17:44:21,en-ZZ,FLAL5IMCB1001,cluster1-FLAL09-item-2,1,...,2008,0,1,1,196,QCY,Cyprus,196,1,1
17,17.0,11960044002,1.720221e+12,FLA-L-9,2024-04-03 17:19:28,2024-04-03 17:44:21,en-ZZ,FLAL5IMCB1001,cluster1-FLAL09-item-2,1,...,2008,0,1,1,196,QCY,Cyprus,196,1,1


In [22]:
df_summ_config = {}

conditions = [
    df_check['db_score_code'].eq('1'),
    df_check['db_score_code'].eq('0'),
    df_check['db_score_code'].eq('9'),
]
codes = [
    1,0,0
]

df_check_sum_score = df_check.copy(deep = True)
df_check_sum_score['score_check'] = np.select(conditions,codes,None)

df_check_sum_score = df_check_sum_score.groupby(['username','unit_id','score']).agg({'score_check':sum}).reset_index(inplace=False)
df_check_sum_score = df_check_sum_score[~df_check_sum_score['unit_id'].isin(cbk.loc[cbk['resp_cat'].str.contains('gap',na=False)].unit_id.unique().tolist())]
df_check_sum_score[['score','score_check']] = df_check_sum_score[['score','score_check']].apply(pd.to_numeric)

df_summ_config["df_check_sum_score"] = "config_check_sum_score"

In [23]:
from test.data_quality.DataQuality import DataQuality
from test.utils.utils import create_df_from_dq_results

df_summ_tab = []

for k, v in df_summ_config.items():
    dq = DataQuality(globals()[k],config_path=f"./test/config/config.json")
    dq_results = dq.run_test()
    dq_table = create_df_from_dq_results(dq_results=dq_results).assign(table=k)
    cols = dq_table.columns.to_list()
    cols = cols[-1:] + cols[:-1]

    df_summ_tab.append(dq_table[cols])

dq_table_all = pd.concat(df_summ_tab,axis = 0)
dq_table_all.head()

,table,column,dimension,status,expectation_type,unexpected_count,element_count,unexpected_percent,percent
0,df_check_sum_score,score; score_check,Validity,PASSED,expect_column_pair_values_to_be_equal,0,388205,0.0,100.0
1,df_check_sum_score,username,Completeness,PASSED,expect_column_values_to_not_be_null,0,388205,0.0,100.0
2,df_check_sum_score,username,Completeness,PASSED,expect_column_value_lengths_to_equal,0,388205,0.0,100.0
3,df_check_sum_score,score,Completeness,PASSED,expect_column_values_to_not_be_null,0,388205,0.0,100.0


In [24]:
val_vars = cbk.loc[~cbk['resp_cat'].str.startswith('gap',na=False),:].qtiLabel2.to_list()

df_sql_check = df_long.loc[
    (~pd.isnull(df_long['qtiLabel'])) & (df_long['qtiLabel'].isin(val_vars)) & (~df_long['qtiLabel'].str.endswith('T',na = False)),
    ['login','unit_id','itemId','qtiLabel']
].assign(dat='1').sort_values(['login','qtiLabel']).merge(
    df_long_check[['login','qtiLabel','source']].assign(sql='1'),
    how = 'outer',
    on = ['login','qtiLabel']
)

conditions = [
    df_sql_check['dat'].eq('1') & df_sql_check['sql'].eq('1'),
    df_sql_check['dat'].eq('1') & ~df_sql_check['sql'].eq('1'),
    ~df_sql_check['dat'].eq('1') & df_sql_check['sql'].eq('1'),
]

codes = [
    'match',
    'dat',
    'sql'
]

df_sql_check['source'] = np.select(conditions,codes,'')
df_sql_check.drop(columns = ['sql','dat'],inplace=True)

df_sql_check.loc[df_sql_check['source'] != 'match',:].login.unique()

array(['A11000035006', 'A12500007032', 'A13000111006', 'A13760082049',
       'A13760150001'], dtype=object)

In [25]:
df_long.loc[df_long['in_cq']=='1',:].groupby(['qtiLabel','score_code','isoalpha3']).size().unstack(fill_value=0).to_excel('./data/FLA_freq_Score_byCnt.xlsx')
df_long.loc[df_long['in_cq']=='1',:].groupby(['qtiLabel','score_code']).size().unstack(fill_value=0).to_excel('./data/FLA_freq_Score_Overall.xlsx')
df_long.loc[df_long['in_cq']=='1',:].groupby(['qtiLabel','cq_cat','isoalpha3']).size().unstack(fill_value=0).to_excel('./data/FLA_freq_Resp_byCnt.xlsx')
df_long.loc[df_long['in_cq']=='1',:].groupby(['qtiLabel','cq_cat']).size().unstack(fill_value=0).to_excel('./data/FLA_freq_Resp_Overall.xlsx')

In [81]:
df_longx = df_long.copy(deep = True)

In [82]:
import re
df_longx['ppart1'] = df_longx['ppart1'].astype(str).apply(lambda x: re.sub(".0","",x)).replace('nan','')
df_longx['mpop1'] = df_longx['mpop1'].astype(str).apply(lambda x: re.sub(".0","",x)).replace('nan','')
df_longx = df_longx.loc[:,~df_longx.columns.duplicated()]

In [86]:
from etl.load import *

make_long_file(df_longx, domain = 'FLA')
make_wide_file(df_longx, cbk = cbk)